In [1]:
import great_expectations as ge
import pandas as pd
import utils
import yattag
import ipywidgets as widgets

# Load data

In [2]:
bbt = utils.load_sheet("budget_by_type")
bt = utils.load_sheet("budget_type")
rt = utils.load_sheet("resource_type")
rbt = utils.load_sheet("resource")

In [3]:
expenses = utils.merge(
    hierarchy_df=bt,
    hierarchy_df_on="id",
    values_df=bbt,
    values_df_on="type",
    hierarchy_suffix="_bt",
    values_suffix="_bbt",
    drop_cols=[
        "name_fr",
        "name_en",
        "name_ar",
        "name",
        "description",
        "id_bt",
        "parent_id",
        "type",
        "id_bbt",
        "organization",
    ],
)
revenues = utils.merge(
    hierarchy_df=rt,
    hierarchy_df_on="id",
    values_df=rbt,
    values_df_on="type",
    hierarchy_suffix="_rt",
    values_suffix="_rbt",
    drop_cols=[
        "name_fr",
        "name_en",
        "name_ar",
        "description",
        "description_fr",
        "description_ar",
        "description_en",
        "level",
        "id_rt",
        "parent_id",
        "id_rbt",
    ],
)

# Expenses

## All ministries have top level budget type

Check the number of unique ministries per year, compare it to the number of ministries with a top level budget type

In [9]:
for year in expenses.year.unique():
    print(year)
    for i in utils.top_level_budget(expenses, year):
        if i not in ['الدولة', 'الصندوق العام للتعويـــــض']:
            print(i)

2015
وزارة التكوين المهني و التشغيل
2016
2017
وزارة المرأة و الأسرة و الطفولة
2018
وزارة الشؤون الإجتماعية
2019


## State budget must equal the sum of ministries' budgets and state-level expenses

For each year, the sum of ministries budgets and نفقات طارئة و غير موزعة and الدين العمومي must equal ميزانية الدولة

In [ ]:
utils.summed_state_budget(expenses, 2019)

In [ ]:
utils.state_budget(expenses, 2019)

In [ ]:
expenses[(expenses.year == 2015) & (expenses.organization_name == "الدولة")]

In [11]:
gap = utils.budget_gap(expenses)
res = gap[(gap.gap != 0) & (gap.double != 2)].set_index(["year", "organization_name"])

In [12]:
res

extra  value_agg parent_name_typed  \
year organization_name                                                    
2015 وزارة التشغيل و التكوين المهني      0     14.900               NaN   
     وزارة التنمية و التعاون الدولي      0    802.012    نفقات التنمية    
2016 وزارة الداخلية                      0    383.300    نفقات التنمية    
     وزارة العدل                         0    432.352               NaN   
2018 رئاسة الحكومة                       0    157.881               NaN   
     هيئة الحقيقة والكرامة               0      0.000    نفقات التنمية    
     هيئة الحقيقة والكرامة               0      0.000    نفقات التنمية    
     هيئة الحقيقة والكرامة               0      0.000               NaN   
     وزارة الشؤون الاجتماعية             0   1138.193               NaN   
     وزارة الشؤون الاجتماعية             0     41.565               NaN   
2019 مجلس نواب الشعب                     0     30.793               NaN   

                                     value_typed       budget_type_name  \
year organization_name                                                    
2015 وزارة التشغيل و التكوين المهني       16.570         نفقات التنمية    
     وزارة التنمية و التعاون الدولي      416.006        التمويل العمومي   
2016 وزارة الداخلية                      113.300        التمويل العمومي   
     وزارة العدل                         432.379           نفقات التصرف   
2018 رئاسة الحكومة                       157.872           نفقات التصرف   
     هيئة الحقيقة والكرامة                   NaN  الاسنثمارات المباشرة    
     هيئة الحقيقة والكرامة                   NaN        التمويل العمومي   
     هيئة الحقيقة والكرامة                   NaN         نفقات التنمية    
     وزارة الشؤون الاجتماعية            1298.364           نفقات التصرف   
     وزارة الشؤون الاجتماعية              50.000         نفقات التنمية    
2019 مجلس نواب الشعب                      31.923           نفقات التصرف   

                                         gap  double  
year organization_name                                
2015 وزارة التشغيل و التكوين المهني    1.670   1.112  
     وزارة التنمية و التعاون الدولي -386.006   1.928  
2016 وزارة الداخلية                 -270.000   3.383  
     وزارة العدل                       0.027   1.000  
2018 رئاسة الحكومة                    -0.009   1.000  
     هيئة الحقيقة والكرامة               NaN     NaN  
     هيئة الحقيقة والكرامة               NaN     NaN  
     هيئة الحقيقة والكرامة               NaN     NaN  
     وزارة الشؤون الاجتماعية         160.171   1.141  
     وزارة الشؤون الاجتماعية           8.435   1.203  
2019 مجلس نواب الشعب                   1.130   1.037